# 17/05 - Pair Programming ETL 1 - Lidia y Tatiana

Tendréis que usar el csv attacks_limpieza_completa.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD.
 
En este momento tenemos dos fuentes de datos:

- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora. Usad el que no tenía las variables categóricas codificadas.
- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer:

>USA

>Australia

>New Zealand

>South Africa

>Papua New Guinea

In [133]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [134]:
# Cargamos el primer archivo
df = pd.read_csv("./attacks_limpieza_completa.csv", index_col = 0)
df.reset_index(inplace=True)

In [135]:
df.head(2)

,index,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,...,1,0,0,1,0,0,4,7,1,9.116327
1,1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,...,1,0,0,1,0,0,3,7,7,3.804860


In [136]:
# Del primer archivo seleccionamos solo los paises que nos interesan
df_paises= df[df["country"].isin(["usa","australia","new zealand", "south africa", "papua new guinea"])]

In [137]:
# Cargamos el segundo archivo
df_clima= pd.read_csv("./df_completo.csv", index_col = 0)
df_clima.reset_index(inplace=True)

Del dataframe de los datos climáticos seleccionaremos todas las columnas.

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por pais.

Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. 

Si intentamos hacer la media de eso no nos dará un valor real. 

Como separar esta información en distintas columnas fue un problema al que nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. 

Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [138]:
df_clima.head(2)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 245, 'speed':...",21,2,8,1017,none,0,0,200,2,USA,39.78373,-100.445882
1,1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 250, 'speed':...",21,2,8,1018,none,0,0,230,2,USA,39.78373,-100.445882


In [139]:
import ast

In [140]:
df_clima["rh_profile"] =df_clima["rh_profile"].apply(ast.literal_eval)

In [141]:
df_clima.rh_profile[0] 

[{'layer': '950mb', 'rh': 5},
 {'layer': '900mb', 'rh': 4},
 {'layer': '850mb', 'rh': 4},
 {'layer': '800mb', 'rh': 4},
 {'layer': '750mb', 'rh': 3},
 {'layer': '700mb', 'rh': 1},
 {'layer': '650mb', 'rh': 2},
 {'layer': '600mb', 'rh': 3},
 {'layer': '550mb', 'rh': 4},
 {'layer': '500mb', 'rh': 5},
 {'layer': '450mb', 'rh': 1},
 {'layer': '400mb', 'rh': 0},
 {'layer': '350mb', 'rh': 3},
 {'layer': '300mb', 'rh': 2},
 {'layer': '250mb', 'rh': 3},
 {'layer': '200mb', 'rh': 8}]

In [142]:
rh= df_clima["rh_profile"].apply(pd.Series)

In [143]:
for i in range(len(rh.columns)):
    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    valores = list(rh[i].apply(pd.Series)["rh"])#tenemos que convertirlo a lista para q funcione el insert
    df_clima.insert(i, nombre, valores)#insert acepta listas

In [144]:
df_clima.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,5,4,4,4,3,1,2,3,4,5,...,8,1017,none,0,0,200,2,USA,39.78373,-100.445882
1,5,4,4,4,1,0,2,3,4,3,...,8,1018,none,0,0,230,2,USA,39.78373,-100.445882


In [145]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [146]:
df_clima['wind_profile'][0] 

[{'layer': '950mb', 'direction': 245, 'speed': 2},
 {'layer': '900mb', 'direction': 230, 'speed': 3},
 {'layer': '850mb', 'direction': 240, 'speed': 3},
 {'layer': '800mb', 'direction': 255, 'speed': 2},
 {'layer': '750mb', 'direction': 265, 'speed': 3},
 {'layer': '700mb', 'direction': 265, 'speed': 3},
 {'layer': '650mb', 'direction': 255, 'speed': 3},
 {'layer': '600mb', 'direction': 250, 'speed': 4},
 {'layer': '550mb', 'direction': 260, 'speed': 5},
 {'layer': '500mb', 'direction': 270, 'speed': 5},
 {'layer': '450mb', 'direction': 285, 'speed': 4},
 {'layer': '400mb', 'direction': 290, 'speed': 5},
 {'layer': '350mb', 'direction': 290, 'speed': 5},
 {'layer': '300mb', 'direction': 300, 'speed': 6},
 {'layer': '250mb', 'direction': 305, 'speed': 5},
 {'layer': '200mb', 'direction': 285, 'speed': 5}]

In [147]:
direction= df_clima["wind_profile"].apply(pd.Series)

In [148]:
for i in range(len(direction.columns)):
    nombre = "wind_" + str(direction[i].apply(pd.Series)["layer"][0]) 
    valores = list(direction[i].apply(pd.Series)["direction"])#tenemos que convertirlo a lista para q funcione el insert
    df_clima.insert(i, nombre, valores)#insert acepta listas

In [149]:
speed= df_clima["wind_profile"].apply(pd.Series)

In [150]:
for i in range(len(speed.columns)):
    nombre = "wind_speed_" + str(speed[i].apply(pd.Series)["layer"][0]) 
    valores = list(speed[i].apply(pd.Series)["speed"])#tenemos que convertirlo a lista para q funcione el insert
    df_clima.insert(i, nombre, valores)#insert acepta listas

In [151]:
df_clima.columns #verificamos que todas nuestras columnas esta creadas

Index(['wind_speed_950mb', 'wind_speed_900mb', 'wind_speed_850mb',
       'wind_speed_800mb', 'wind_speed_750mb', 'wind_speed_700mb',
       'wind_speed_650mb', 'wind_speed_600mb', 'wind_speed_550mb',
       'wind_speed_500mb', 'wind_speed_450mb', 'wind_speed_400mb',
       'wind_speed_350mb', 'wind_speed_300mb', 'wind_speed_250mb',
       'wind_speed_200mb', 'wind_950mb', 'wind_900mb', 'wind_850mb',
       'wind_800mb', 'wind_750mb', 'wind_700mb', 'wind_650mb', 'wind_600mb',
       'wind_550mb', 'wind_500mb', 'wind_450mb', 'wind_400mb', 'wind_350mb',
       'wind_300mb', 'wind_250mb', 'wind_200mb', 'rh_950mb', 'rh_900mb',
       'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb',
       'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb',
       'rh_250mb', 'rh_200mb', 'index', 'timepoint', 'cloudcover', 'highcloud',
       'midcloud', 'lowcloud', 'rh_profile', 'wind_profile', 'temp2m',
       'lifted_index', 'rh2m', 'msl_pressure', 'prec_type', 

Sacamos las medias de la tabla de clima por pais

In [152]:
df_clima_agrupado= df_clima.groupby("pais").mean()

In [153]:
df_clima_agrupado

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
pais,,,,,,,,,,,,,,,,,,,,,
Australia,3.125000,2.921875,2.812500,2.640625,2.671875,2.781250,2.859375,3.093750,3.187500,3.359375,...,24.093750,0.37500,10.796875,1019.156250,3.484375,0.0,133.437500,2.984375,-24.776109,134.755000
New Zealand,5.171875,5.281250,5.375000,5.515625,5.625000,5.859375,6.078125,6.343750,6.703125,7.093750,...,12.781250,3.09375,10.890625,1006.625000,3.671875,0.0,255.000000,4.500000,-41.500083,172.834408
Papua New Guinea,3.468750,3.421875,3.125000,2.781250,2.281250,2.203125,2.437500,2.609375,2.671875,3.031250,...,26.578125,-0.53125,10.453125,1012.546875,1.921875,0.0,100.546875,3.140625,-5.681607,144.248908
South Africa,3.859375,4.000000,4.046875,4.265625,4.531250,4.859375,5.218750,5.546875,5.890625,6.437500,...,20.000000,1.90625,10.296875,1016.703125,6.656250,0.0,210.312500,3.468750,-28.816624,24.991639
USA,2.375000,2.921875,3.281250,3.453125,3.531250,3.593750,3.781250,4.046875,4.171875,4.218750,...,22.609375,1.90625,5.031250,1015.468750,0.000000,0.0,136.093750,2.203125,39.783730,-100.445882


In [154]:
df_clima_agrupado= pd.DataFrame(df_clima_agrupado.reset_index())

In [155]:
df_clima_agrupado.head()

,pais,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,Australia,3.125000,2.921875,2.812500,2.640625,2.671875,2.781250,2.859375,3.093750,3.187500,...,24.093750,0.37500,10.796875,1019.156250,3.484375,0.0,133.437500,2.984375,-24.776109,134.755000
1,New Zealand,5.171875,5.281250,5.375000,5.515625,5.625000,5.859375,6.078125,6.343750,6.703125,...,12.781250,3.09375,10.890625,1006.625000,3.671875,0.0,255.000000,4.500000,-41.500083,172.834408
2,Papua New Guinea,3.468750,3.421875,3.125000,2.781250,2.281250,2.203125,2.437500,2.609375,2.671875,...,26.578125,-0.53125,10.453125,1012.546875,1.921875,0.0,100.546875,3.140625,-5.681607,144.248908
3,South Africa,3.859375,4.000000,4.046875,4.265625,4.531250,4.859375,5.218750,5.546875,5.890625,...,20.000000,1.90625,10.296875,1016.703125,6.656250,0.0,210.312500,3.468750,-28.816624,24.991639
4,USA,2.375000,2.921875,3.281250,3.453125,3.531250,3.593750,3.781250,4.046875,4.171875,...,22.609375,1.90625,5.031250,1015.468750,0.000000,0.0,136.093750,2.203125,39.783730,-100.445882


In [156]:
df_clima_agrupado.columns 

Index(['pais', 'wind_speed_950mb', 'wind_speed_900mb', 'wind_speed_850mb',
       'wind_speed_800mb', 'wind_speed_750mb', 'wind_speed_700mb',
       'wind_speed_650mb', 'wind_speed_600mb', 'wind_speed_550mb',
       'wind_speed_500mb', 'wind_speed_450mb', 'wind_speed_400mb',
       'wind_speed_350mb', 'wind_speed_300mb', 'wind_speed_250mb',
       'wind_speed_200mb', 'wind_950mb', 'wind_900mb', 'wind_850mb',
       'wind_800mb', 'wind_750mb', 'wind_700mb', 'wind_650mb', 'wind_600mb',
       'wind_550mb', 'wind_500mb', 'wind_450mb', 'wind_400mb', 'wind_350mb',
       'wind_300mb', 'wind_250mb', 'wind_200mb', 'rh_950mb', 'rh_900mb',
       'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb',
       'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb',
       'rh_250mb', 'rh_200mb', 'index', 'timepoint', 'cloudcover', 'highcloud',
       'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_amount', 'snow_depth', 'wind1

In [157]:
df_paises.shape

(4335, 22)

In [158]:
df_clima_agrupado.shape

(5, 65)

In [159]:
def minuscula(col):
    try:
        col=col.lower
    except:
        pass
    return col


In [160]:
df_clima_agrupado["pais"] = df_clima_agrupado["pais"].apply(minuscula)

In [161]:
df_unido_total = pd.merge(df_paises,df_clima_agrupado, left_on=["country"], right_on=["pais"])

In [162]:
df_paises.columns

Index(['index', 'year', 'type', 'country', 'age', 'species_', 'fecha_limpia',
       'fatal', 'sex', 'latitud', 'longitud', 'country2', 'fatal_N',
       'fatal_Unknown', 'fatal_Y', 'fatal_N.1', 'fatal_Unknown.1', 'fatal_Y.1',
       'species_.1', 'fecha_limpia.1', 'type.1', 'age_NORM'],
      dtype='object')

In [163]:
df_paises.country.unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [164]:
df_clima_agrupado.pais.unique()

array([<built-in method lower of str object at 0x7faca2a25770>,
       <built-in method lower of str object at 0x7faca2a1e170>,
       <built-in method lower of str object at 0x7facd82e8260>,
       <built-in method lower of str object at 0x7faca2a1ed70>,
       <built-in method lower of str object at 0x7faca2a25c70>],
      dtype=object)

Como tenemos los nombres de los países escritos distinto, los cambiamos

In [165]:
df_unido_total.head(4)

,index_x,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_y,longitud_y


Guardar los resultados obtenidos en un csv.

In [166]:
df_unido_total.to_csv("df_paises_agrupados_media.csv")